In [ ]:
#lane detection code using canny and hough transfrom 
    #(input:video recorded by self driving car)
    #(output:video of detected lanes)
    
    #limiation : detects only edges at region of interest(middle region facing the camera).
    #not robust against sharp road steering or abnormal driving conditions


#Created by Mohamed Ehab and Nayer Nagi

In [1]:
import numpy as np
import math
import cv2
import os
from matplotlib import pyplot as plt




In [2]:
input_vid_dic   = r'../input_videos/'
input_img_dic   = r'../input_imgs/'

output_vid_dic  = r'../output_videos/'


In [3]:
#Load images for tuning algorithms parameters

input_images_list = os.listdir(input_img_dic)
input_images=[]
for img_dic in input_images_list:
    img=cv2.imread( input_img_dic+ img_dic)
    input_images.append(img)

In [4]:
def Canny_edge(img,min_threshold,max_threshold,kernel_size):
    
     #obtain edges using canny algorithm applied on gray scale image filtered by gaussain kernal
    
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    gray = cv2.GaussianBlur(gray,(11, 11),0)
   
    edge=cv2.Canny(gray, min_threshold, max_threshold, apertureSize=kernel_size)
      
    return edge


In [5]:
def region_of_interest(edges):
    # initializing mask with same shape as edges photo
    mask = np.zeros_like(edges)
    imshape = edges.shape[:2]
    
    
    # obtain mask using 4 corners of region of interest
    vertices = np.array([[(150,imshape[0]),(370, 320), (540, 320), (860, imshape[0])]], dtype = np.int32)
    cv2.fillPoly(mask, vertices, 255)
    
    #element wise multiplication
    masked_edges = edges * mask
    return masked_edges

In [6]:
def Hough(edges,rho,theta,threshold,min_length,max_gap):
    lines=cv2.HoughLinesP(edges,rho,theta,threshold,min_length,max_gap)
    return lines

In [7]:
def get_line_param(lines):
    
    
     #obtain slopes and intercepts parameters 
        
    N=len(lines)

    slopes = np.zeros([N,1])
    intc   = np.zeros([N,1])
    L      = np.zeros([N,1])
    i=0
    for i in range(N):
        
        x_1=lines[i,0,0]
        y_1=lines[i,0,1]
        x_2=lines[i,0,2]
        y_2=lines[i,0,3]
        
        L[i]         = math.sqrt((x_2-x_1)**2+(y_2-y_1)**2)
        slopes[i]    = (y_2-y_1)/(x_2-x_1)
        intc[i]      = y_1-slopes[i]*x_1
        
        i+=1
             
    return slopes , intc 

In [8]:

def filter_lines(lines):
    
     #filter out horizontal and vertical lines, and merging lines with similar sopes and intecepts within certian threshold 
     
    #output --> filtered lines in form (Nx4)
    
    
    
    
        
    slope_similarity_threshold = 0.9
    intercept_similarity_threshold = 70
    min_slope_threshold = 0.1
    
    slopes, intercepts = get_line_param(lines)
    
    filtered_slopes=[]
    filtered_intercepts=[]

    
    
    #cluster of indices of similar lines to be averaged for e.g [[1,8,16],[.,.],....] ---> lines 1,8 and 16 are similar.
    index_cluster=[]
    
    
    temp=[]
    
    #copy slopes
    temp_slopes=slopes
    for i in range(len(slopes)):
        slope=temp_slopes[i]
        
        #filter out verctical and horizontal lines
        if np.abs(slope) > min_slope_threshold and lines[i,0,3] != lines[i,0,1] and np.abs(slope) < float('Inf')  :
                intc=intercepts[i] 
                
                #add one index line to the cluster
                temp.append(i)
                
                
                
                 #loop to search for lines similar to that added in the cluster
                for j in range(len(slopes)):

                    slope_2=temp_slopes[j]

                    if np.abs(slope_2) > min_slope_threshold :

                        intc_2=intercepts[j]
                        
                        

                        if (j != i and np.abs(slope-slope_2)<slope_similarity_threshold and np.abs(intc-intc_2)<intercept_similarity_threshold ):
                            
                            
                            temp.append(j)
                            
                #set the slope of the lines similar to the first added line to 0 
                #(to filter it out at horiznal lines check (to prevent checking it more than one time))
                            temp_slopes[j]=0

                index_cluster.append(temp[:])
                temp.clear()

    
    final_lines=np.zeros([len(index_cluster),4])
    slope=0
    intc=0
    for i in range(len(index_cluster)):
        
        N=len(index_cluster[i])
        
        for j in range(N):
            
            final_lines[i,0]=final_lines[i,0]+lines[index_cluster[i][j],0,0]/N
            final_lines[i,1]=final_lines[i,1]+lines[index_cluster[i][j],0,1]/N
            final_lines[i,2]=final_lines[i,2]+lines[index_cluster[i][j],0,2]/N
            final_lines[i,3]=final_lines[i,3]+lines[index_cluster[i][j],0,3]/N
            

            
            
            


        

    return final_lines  

In [9]:
def draw_lines(points_list,image):
    
   
    img=np.copy(image)
    y_max=image.shape[0]
    
    for x_1,y_1,x_2,y_2 in points_list.astype(int): 
        m=(y_2-y_1)/(x_2-x_1)
        c= y_1-m*x_1
        
        theta=np.arctan2(y_2-y_1,x_2-x_1)
        L=abs(y_max/(2.6*math.sin(theta)))

        x_max=int((y_max-c)/m)

        y_min=int(y_max-np.sign(theta)*L*np.sin(theta))

        x_min=int((y_min-c)/m)
        
        
        img = cv2.line(img, (x_max,y_max), (x_min,y_min), (0, 255, 0), thickness=3, lineType=8)
        
    return img

In [10]:
def process(image_rgb):
    
 #get edges using canny algorithm
    edges = Canny_edge(image_rgb , 100 , 150 , 3)
    
 #get edges at region of interest 
    masked_edges= region_of_interest(edges)
 
 #obtain lines using hough transfrom
    lines = Hough( masked_edges,1,np.pi/100,8,200,22)
 
 #filter lines
    lines_filtered = filter_lines(lines)
   
 #get image with drawn lines
    final_img=draw_lines(lines_filtered,image_rgb)
    
    return final_img


    
    

In [16]:
test_vid = os.listdir(input_vid_dic)

clip = cv2.VideoCapture(input_vid_dic + test_vid[1])
fourcc = cv2.VideoWriter_fourcc(*'XVID')

out = cv2.VideoWriter(output_vid_dic+'solidYellowLeft.avi',fourcc, 60,(int(clip.get(3)),int(clip.get(4))))


    


while clip.isOpened():
    
    flag,frame  = clip.read()
    
    if not flag:
            break
        
    processed_frame = process(frame)
    out.write(processed_frame)
 
    
    
out.release()
    
clip.release()   

cv2.destroyAllWindows() 
